In [2]:
import os, sys
sys.path.insert(0, '/home/jtorrenc/mvesc/ETL')
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import random
from functools import partial

In [3]:
def color_gt(val,x=.5,c='blue',neg=True):
    """
    Colors values greater than x blue (or other provided color)
    
    :param float x: lower bound on colored values
    :param str c: color
    :param bool neg: if true, color negative values red
    :returns: css string to color values in dataframe
    :rtype: str
    """
    
    if val > x:
        color = c 
    elif val < 0:
        color = 'red' 
    else: color = 'black'
    return 'color: %s' % color

def table_exists(cursor, table, schema='clean'):
    """
    Checks to see if a table exists in the database
    
    :param pg.cursor cursor: 
    :param str table:
    :param str schema:
    :rtype: bool
    """
    cursor.execute("""                                                   
            select count(*) from information_schema.tables                       
            where table_schema = %s and table_name = %s
            """, [schema, table])
    return cursor.fetchall()[0][0]

In [18]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        best = dict()
        for m in model_list:
            cursor.execute("""select filename from model.reports where model_name = %s
                            order by val_precision_10 desc limit 1; """, [m])
            best[m] = cursor.fetchall()[0][0]
    predictions = pd.DataFrame()
    for m, f in best.items():
        df = read_table_to_df(connection, f,columns = ['student_lookup','predicted_score'],
                                          schema='predictions',nrows=-1)
        df.set_index('student_lookup',inplace=True)
        df.columns = [m]
        predictions = pd.concat((predictions,df), axis=1, join='inner')


In [19]:
spearman_pred = predictions.corr('spearman')

# Spearman's Prediction Correlation

In [20]:
spearman_pred.style.applymap(partial(color_gt,x=.75))

In [54]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        cursor.execute('select distinct feature_categories from model.reports')
        feature_list = [f[0] for f in cursor.fetchall()]
        for f in feature_list:  #this won't work anymore when there are other combinations of features
            if ',' in f:
                feature_list.remove(f)
                if 'all' not in feature_list:
                    feature_list.append('all')
        best = dict()
        features = dict()
        spearman = dict()
        for f in feature_list:
            best[f] = dict()
            for m in model_list:
                if f != 'all':
                    cursor.execute("""select filename from model.reports 
                                    where model_name = %s
                                    and feature_categories = %s
                                    and feature_grades = '5, 6, 7, 8, 9'
                                    order by val_precision_10 desc limit 1; """, [m,f])
                else:
                    cursor.execute("""select filename from model.reports 
                                    where model_name = '{}'
                                    and feature_categories like '%,%'
                                    and feature_grades = '5, 6, 7, 8, 9'
                                    order by val_precision_10 desc limit 1; """.format(m))
                filename = cursor.fetchall()
                best[f][m] = filename[0][0]
            temp = pd.DataFrame()
            for m, file in best[f].items():
                if table_exists(cursor, file, 'feature_scores'):
                    df = read_table_to_df(connection, file ,columns=['feature','importance'],
                                          schema='feature_scores',nrows = -1)
                    df.set_index('feature',inplace=True)
                    df.columns = [m]
                    temp = pd.concat((temp,df), axis=1, join='outer')
            features[f] = temp
            spearman[f] = temp.abs().corr('spearman')

In [55]:
features.keys()

dict_keys(['all', 'oaa_normalized', 'absence', 'snapshots', 'grades', 'mobility'])

In [56]:
best['mobility']

{'AB': 'param_set_453_AB_ht_4524',
 'DT': 'param_set_458_DT_ht_4571',
 'ET': 'param_set_453_ET_ht_4523',
 'GB': 'param_set_468_GB_ht_4676',
 'KNN': 'param_set_458_KNN_ht_4579',
 'NB': 'param_set_463_NB_ht_4627',
 'RF': 'param_set_463_RF_ht_4622',
 'SGD': 'param_set_453_SGD_ht_4528',
 'SVM': 'param_set_453_SVM_ht_4525',
 'logit': 'param_set_453_logit_ht_4520'}

# Spearman's Feature Correlation


In [57]:
features['absence'].head()

,RF,SVM,GB,AB,logit,ET,SGD,DT
absence_consec_gr_7,0.000789,0.087852,0.000000,0.000000,0.262957,0.000924,22.639499,0.000000
absence_consec_gr_8,0.028656,-0.000016,0.011905,0.003216,0.031852,0.028532,1.331735,0.000000
absence_consec_gr_9,0.046819,0.000055,0.060845,0.099215,0.269176,0.046311,9.322147,0.040512
absence_gr_5,0.064056,-0.000023,0.078696,NaN,-0.046104,0.073802,-8.101389,0.020813
absence_gr_6,0.079172,0.000008,0.081279,0.052965,-0.222509,0.079043,-9.943623,0.127702


In [58]:
spearman['absence'].style.applymap(partial(color_gt,x=.7))

In [59]:
features['mobility'].head()

,RF,SVM,GB,AB,logit,ET,SGD,DT
avg_address_change_to_gr_5,0.002644,0.418592,0.006284,0.0,0.000000,0.001653,1.331735e+00,0.000000
avg_address_change_to_gr_5_isnull,0.004189,0.280750,0.000010,0.0,0.041814,0.002890,-1.512494e-14,0.007397
avg_address_change_to_gr_6,0.005937,0.071333,0.000757,0.0,0.000000,0.004239,5.553688e-15,0.000000
avg_address_change_to_gr_6_isnull,0.002541,-0.000069,0.000000,0.0,0.005640,0.002361,-1.276167e-14,0.000000
avg_address_change_to_gr_7,0.004179,-0.014458,0.008888,0.0,0.000000,0.003427,-2.219559e-01,0.000000


In [60]:
spearman['mobility'].style.applymap(partial(color_gt,x=.7))

In [61]:
features['oaa_normalized'].head()

,RF,SVM,GB,AB,logit,ET,SGD,DT
eighth_math_normalized,0.039856,0.085535,0.038644,0.127352,0.000000,0.021150,2.138664,0.000000
eighth_math_normalized_isnull,0.006814,0.169345,0.000000,0.000000,0.000000,0.004572,5.326941,0.000000
eighth_math_percentile,0.034348,-0.055547,0.018720,0.044069,0.000000,0.035541,-2.593598,0.080650
eighth_math_percentile_isnull,0.006687,0.169345,0.004537,0.000000,0.000000,NaN,5.326941,0.025544
eighth_math_pl_Accelerated,0.002173,-0.008385,0.000000,0.000000,-0.233912,0.005935,-2.663471,0.000000


In [62]:
spearman['oaa_normalized'].style.applymap(partial(color_gt,x=.7))

In [63]:
features['snapshots'].head()


,RF,SVM,GB,AB,logit,ET,SGD,DT
days_absent_excused_gr_7,0.026768,-0.713126,0.014806,0.023076,-0.505254,0.019405,-156.283049,0.000000
days_absent_excused_gr_8,0.023392,0.100783,0.010918,0.000000,0.000000,0.022353,17.032193,0.000000
days_absent_excused_gr_9,0.069043,0.394523,0.024187,0.000000,0.351827,0.057544,-0.003329,0.032535
days_absent_gr_5,0.020311,-0.063759,0.022970,0.000000,-0.061790,0.016444,-159.031385,0.017877
days_absent_gr_6,0.019935,-0.036667,0.017842,0.014429,-0.096762,0.016859,-145.958184,0.000000


In [64]:
spearman['snapshots'].style.applymap(partial(color_gt,x=.7))

In [65]:
features['grades'].head()

,RF,SVM,GB,AB,logit,ET,SGD,DT
art_gpa_gr_5,0.004507,-0.215649,0.000000,0.032600,-0.932123,0.003461,-1.832721,0.023892
art_gpa_gr_5_isnull,0.000518,0.255441,0.000000,0.000000,0.000000,0.001457,-19.976029,0.000000
art_gpa_gr_6,0.009637,0.211025,0.005868,0.025582,0.814532,0.005430,45.837587,0.000000
art_gpa_gr_6_isnull,0.001762,-0.830613,0.000000,0.000000,-0.977115,0.006097,-30.629911,0.000000
art_gpa_gr_7,0.011735,0.055578,0.004427,0.027485,0.093914,0.010712,33.371786,0.030191


In [66]:
spearman['grades'].style.applymap(partial(color_gt,x=.7))

In [67]:
features['all'].head()

,RF,SVM,GB,AB,logit,ET,SGD,DT
absence_consec_gr_7,0.000000,0.141226,0.000000,0.000000,0.249384,0.000000,90.557997,0.0
absence_consec_gr_8,0.001153,0.015964,0.006129,0.000000,0.027541,0.001173,194.433347,0.0
absence_consec_gr_9,0.006888,0.017218,0.012071,0.017715,0.142771,0.002286,857.637502,0.0
absence_gr_5,0.002341,0.007709,0.002997,0.000000,-0.014128,0.001349,-80.902917,0.0
absence_gr_6,0.003860,0.025194,0.012652,0.000000,-0.063706,0.004162,-94.730768,0.0


In [68]:
spearman['all'].style.applymap(partial(color_gt,x=.7))

In [69]:
pd.Panel(spearman).mean(axis=0).style.applymap(partial(color_gt,x=.7,c='blue'))

In [ ]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute("""select, """ 